In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from neural_network import NeuralNetworkModel, ChessDataset, ChessEvaluationNeuralNetwork

In [3]:
df = pd.read_csv("../../data/chess_data.csv")

df = df[~df['Evaluation'].str.contains('#', na=False)]
X = df['FEN'].apply(NeuralNetworkModel.fen_to_feature_array).tolist()
X = np.array(X, dtype=np.float32)
y = df['Evaluation'].apply(lambda x: float(x)).astype(np.float32).values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


train_dataset = ChessDataset(X_train, y_train)
test_dataset = ChessDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [4]:
model = ChessEvaluationNeuralNetwork(input_size=X.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(100):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")

Epoch 1, Loss: 367420.5618
Epoch 2, Loss: 345893.3333
Epoch 3, Loss: 326947.2408
Epoch 4, Loss: 310707.1357
Epoch 5, Loss: 299233.4750
Epoch 6, Loss: 290878.1974
Epoch 7, Loss: 283247.3704
Epoch 8, Loss: 273944.6698
Epoch 9, Loss: 266409.1027
Epoch 10, Loss: 262375.6890
Epoch 11, Loss: 254384.7618
Epoch 12, Loss: 248943.4825
Epoch 13, Loss: 242930.0842
Epoch 14, Loss: 236099.4430
Epoch 15, Loss: 233670.7819
Epoch 16, Loss: 226614.2765
Epoch 17, Loss: 221269.2277
Epoch 18, Loss: 216416.2884
Epoch 19, Loss: 212447.2408
Epoch 20, Loss: 207124.8049
Epoch 21, Loss: 206207.6170
Epoch 22, Loss: 201129.4538
Epoch 23, Loss: 192556.4602
Epoch 24, Loss: 191095.7385
Epoch 25, Loss: 186954.2691
Epoch 26, Loss: 184218.5067
Epoch 27, Loss: 183174.3129
Epoch 28, Loss: 175478.5289
Epoch 29, Loss: 174582.9381
Epoch 30, Loss: 168840.6397
Epoch 31, Loss: 168121.1887
Epoch 32, Loss: 163207.7072
Epoch 33, Loss: 161149.1781
Epoch 34, Loss: 156530.6353
Epoch 35, Loss: 155449.5973
Epoch 36, Loss: 151474.9002
E

In [5]:
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()

avg_test_loss = test_loss / len(test_loader)
rmse_error = np.sqrt(avg_test_loss)
print(f"\nTest RMSE: {rmse_error:.4f}")


Test RMSE: 488.8287


In [6]:
torch.save(model.state_dict(), 'neural_network.pth')